In [51]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import nltk
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, classification_report
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer, StandardScaler
from sklearn.model_selection import StratifiedKFold, cross_validate
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import StratifiedKFold, cross_val_score
import datasets
import joblib
import os
from sklearn.base import clone
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [52]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [53]:
df = pd.read_csv('/content/drive/MyDrive/Grammar_autocorrection_df1_new.csv')

In [54]:
df.head()

,Error Type,Ungrammatical Statement,Standard English,Ungrammatical Length,Standard Length,Tokenized Ungrammatical,Tokenized Standard,POS_Ungrammatical,POS_Standard,POS_W2V_Ungram,...,W2V_Ungram,W2V_Standard,Bigrams_Ungrammatical,Bigrams_Standard,Trigrams_Ungrammatical,Trigrams_Standard,W2V_Bigrams_Ungrammatical,W2V_Bigrams_Standard,W2V_Trigrams_Ungrammatical,W2V_Trigrams_Standard
0,verb tense errors,i goes to the store everyday,i go to the store everyday,29,27,"['i', 'goes', 'to', 'the', 'store', 'everyday']","['i', 'go', 'to', 'the', 'store', 'everyday']","[('i', 'NN'), ('goes', 'VBZ'), ('to', 'TO'), (...","[('i', 'NN'), ('go', 'VBP'), ('to', 'TO'), ('t...",[-0.09392828 0.01467511 -0.11880406 -0.019369...,...,[-0.26300946 0.39695683 0.13605912 0.053938...,[-0.26916853 0.40635803 0.14320455 0.059786...,"['i goes', 'goes to', 'to the', 'the store', '...","['i go', 'go to', 'to the', 'the store', 'stor...","['i goes to', 'goes to the', 'to the store', '...","['i go to', 'go to the', 'to the store', 'the ...",[-1.4865453e-02 1.1285488e-02 2.3132097e-02 ...,[-0.01412573 0.01116349 0.02157649 0.002338...,[ 0.00053252 -0.00972586 -0.00517551 -0.001490...,[ 2.8302954e-03 -1.1065927e-02 -5.8973590e-03 ...
1,verb tense errors,they was playing soccer last night,they were playing soccer last night,35,36,"['they', 'was', 'playing', 'soccer', 'last', '...","['they', 'were', 'playing', 'soccer', 'last', ...","[('they', 'PRP'), ('was', 'VBD'), ('playing', ...","[('they', 'PRP'), ('were', 'VBD'), ('playing',...",[-0.08057409 -0.064899 -0.22793531 -0.064086...,...,[-0.31353074 0.4317571 0.14008799 0.045958...,[-0.2976561 0.4338199 0.14537303 0.054469...,"['they was', 'was playing', 'playing soccer', ...","['they were', 'were playing', 'playing soccer'...","['they was playing', 'was playing soccer', 'pl...","['they were playing', 'were playing soccer', '...",[-9.9777804e-05 -2.6923325e-04 -1.0527767e-03 ...,[-0.00390252 0.00116026 0.00490098 -0.003110...,[ 3.8320364e-03 -9.6556135e-03 -1.7034016e-03 ...,[ 0.00012496 -0.00871335 -0.00053851 -0.003852...
2,verb tense errors,she have completed her homework,she has completed her homework,32,31,"['she', 'have', 'completed', 'her', 'homework']","['she', 'has', 'completed', 'her', 'homework']","[('she', 'PRP'), ('have', 'VBP'), ('completed'...","[('she', 'PRP'), ('has', 'VBZ'), ('completed',...",[-0.16420007 -0.03099914 -0.17860003 -0.127690...,...,[-2.67656505e-01 3.97760749e-01 1.48411125e-...,[-0.28929004 0.39424703 0.13514736 0.038457...,"['she have', 'have completed', 'completed her'...","['she has', 'has completed', 'completed her', ...","['she have completed', 'have completed her', '...","['she has completed', 'has completed her', 'co...",[-0.01038111 -0.00510946 0.00610128 -0.000997...,[-0.01103961 -0.00731192 -0.00655014 0.003719...,[ 0.00674664 0.00827969 -0.00871949 -0.008137...,[-4.41420777e-03 -4.46065888e-03 1.07493391e-...
3,verb tense errors,he do not know the answer,he does not know the answer,25,27,"['he', 'do', 'not', 'know', 'the', 'answer']","['he', 'does', 'not', 'know', 'the', 'answer']","[('he', 'PRP'), ('do', 'VBZ'), ('not', 'RB'), ...","[('he', 'PRP'), ('does', 'VBZ'), ('not', 'RB')...",[-0.20836382 -0.00077939 -0.24012117 -0.141872...,...,[-0.31522754 0.4564127 0.18422635 0.063359...,[-0.30345047 0.43326005 0.16985984 0.060665...,"['he do', 'do not', 'not know', 'know the', 't...","['he does', 'does not', 'not know', 'know the'...","['he do not', 'do not know', 'not know the', '...","['he does not', 'does not know', 'not know the...",[ 0.01418456 0.00413721 0.00204819 -0.014849...,[ 0.01053579 -0.00204841 0.00280879 -0.013587...,[ 0.00571257 0.00300963 0.00256153 0.005161...,[ 7.0049549e-03 1.2993978e-02 2.4182086e-03 ...
4,verb tense errors,the sun rise in the east,the sun rises in the east,25,26,"['the', 'sun', 'rise', 'in', 'the', 'east']","['the', 'sun', 'rises', 'in', 'the', 'east']","[('the', 'DT'), ('sun', 'NN'), ('rise', 'NN'),...","[('the', 'DT'), ('sun', 'NN'), ('r

In [55]:
# printing dataset columns
print(df.columns)

Index(['Error Type', 'Ungrammatical Statement', 'Standard English',
       'Ungrammatical Length', 'Standard Length', 'Tokenized Ungrammatical',
       'Tokenized Standard', 'POS_Ungrammatical', 'POS_Standard',
       'POS_W2V_Ungram', 'POS_W2V_Standard', 'POS_seq_U', 'POS_seq_S',
       'W2V_Ungram', 'W2V_Standard', 'Bigrams_Ungrammatical',
       'Bigrams_Standard', 'Trigrams_Ungrammatical', 'Trigrams_Standard',
       'W2V_Bigrams_Ungrammatical', 'W2V_Bigrams_Standard',
       'W2V_Trigrams_Ungrammatical', 'W2V_Trigrams_Standard'],
      dtype='object')


# Naive Bayes

In [56]:
embedding_experiments = {
    'POS':     ('POS_W2V_Ungram',             'POS_W2V_Standard'),
    'Unigram': ('W2V_Ungram',                 'W2V_Standard'),
    'Bigram':  ('W2V_Bigrams_Ungrammatical',  'W2V_Bigrams_Standard'),
    'Trigram': ('W2V_Trigrams_Ungrammatical', 'W2V_Trigrams_Standard')
}

In [57]:
# Ensure save directory exists
save_dir = "saved_models"
os.makedirs(save_dir, exist_ok=True)

# Define & apply parser once
def parse_embedding(s):
    if isinstance(s, str):
        return np.array([float(x) for x in s.strip('[]').split()], dtype=float)
    return np.zeros(0, dtype=float)

for ug_col, st_col in embedding_experiments.values():
    df[ug_col] = df[ug_col].apply(parse_embedding)
    df[st_col] = df[st_col].apply(parse_embedding)

# Helper to stack feature arrays
def stack_feats(X):
    return np.vstack(X['feat'])

# Pipeline builder for Naive Bayes
def make_embedding_pipeline_nb(ug_col, st_col):
    n = len(df)
    feats = list(df[ug_col]) + list(df[st_col])
    y     = np.array([0]*n + [1]*n)  # 0=ungrammatical, 1=correct
    X_df  = pd.DataFrame({'feat': feats})

    stacker = FunctionTransformer(stack_feats, validate=False)
    pipe    = Pipeline([
        ('stack', stacker),
        ('clf',   GaussianNB() )
    ])
    return pipe, X_df, y

# Train/Test split evaluation (90/10) & save models
print("=== Train/Test split 90/10 with Naive Bayes ===")
trained_pipelines = {}

for name, (ug, st) in embedding_experiments.items():
    pipe, X_df, y = make_embedding_pipeline_nb(ug, st)

    X_tr, X_te, y_tr, y_te = train_test_split(
        X_df, y,
        test_size=0.1,
        stratify=y,
        random_state=42
    )

    # Train & evaluate
    pipe.fit(X_tr, y_tr)
    preds = pipe.predict(X_te)
    trained_pipelines[name] = pipe

    # Print metrics
    print(f"\n{name} accuracy: {accuracy_score(y_te, preds):.4f}")
    print(classification_report(y_te, preds, digits=4))

    # Save trained pipeline
    model_path = os.path.join(save_dir, f"{name}_nb_pipeline.pkl")
    joblib.dump(pipe, model_path)
    print(f"Saved model to {model_path}")


=== Train/Test split 90/10 with Naive Bayes ===

POS accuracy: 0.5545
              precision    recall  f1-score   support

           0     0.5663    0.4653    0.5109       202
           1     0.5462    0.6436    0.5909       202

    accuracy                         0.5545       404
   macro avg     0.5562    0.5545    0.5509       404
weighted avg     0.5562    0.5545    0.5509       404

Saved model to saved_models/POS_nb_pipeline.pkl

Unigram accuracy: 0.5074
              precision    recall  f1-score   support

           0     0.5062    0.6089    0.5528       202
           1     0.5093    0.4059    0.4518       202

    accuracy                         0.5074       404
   macro avg     0.5077    0.5074    0.5023       404
weighted avg     0.5077    0.5074    0.5023       404

Saved model to saved_models/Unigram_nb_pipeline.pkl

Bigram accuracy: 0.4926
              precision    recall  f1-score   support

           0     0.4946    0.6782    0.5720       202
           1    

In [58]:
# Ensure save directory exists
save_dir = "saved_models"
os.makedirs(save_dir, exist_ok=True)

print("\n=== 10-Fold Cross Validation with Naive Bayes ===")
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

for name, (ug, st) in embedding_experiments.items():
    # Build pipeline and data
    pipe, X_df, y = make_embedding_pipeline_nb(ug, st)
    accs, precs, recs, f1s = [], [], [], []

    # Perform 10-fold CV
    for train_idx, test_idx in cv.split(X_df, y):
        p = clone(pipe)
        X_train, X_test = X_df.iloc[train_idx], X_df.iloc[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        p.fit(X_train, y_train)
        preds = p.predict(X_test)

        accs.append(accuracy_score(y_test, preds))
        precs.append(precision_score(y_test, preds))
        recs.append(recall_score(y_test, preds))
        f1s.append(f1_score(y_test, preds))

    # Calculate mean ± std for each metric
    mean_acc, std_acc = np.mean(accs), np.std(accs)
    mean_prec, std_prec = np.mean(precs), np.std(precs)
    mean_rec, std_rec = np.mean(recs), np.std(recs)
    mean_f1, std_f1 = np.mean(f1s), np.std(f1s)

    # Print summary
    print(
        f"{name} - CV Accuracy: {mean_acc:.4f} ± {std_acc:.4f} | "
        f"Precision: {mean_prec:.4f} ± {std_prec:.4f} | "
        f"Recall: {mean_rec:.4f} ± {std_rec:.4f} | "
        f"F1: {mean_f1:.4f} ± {std_f1:.4f}"
    )

    # Retrain on full dataset and save final model
    pipe.fit(X_df, y)
    model_path = os.path.join(save_dir, f"{name}_CV_nb_pipeline.pkl")
    joblib.dump(pipe, model_path)
    print(f"Saved final CV model to {model_path}")


=== 10-Fold Cross Validation with Naive Bayes ===
POS - CV Accuracy: 0.5223 ± 0.0200 | Precision: 0.5184 ± 0.0161 | Recall: 0.6318 ± 0.0265 | F1: 0.5693 ± 0.0184
Saved final CV model to saved_models/POS_CV_nb_pipeline.pkl
Unigram - CV Accuracy: 0.5141 ± 0.0264 | Precision: 0.5174 ± 0.0335 | Recall: 0.4535 ± 0.0440 | F1: 0.4820 ± 0.0309
Saved final CV model to saved_models/Unigram_CV_nb_pipeline.pkl
Bigram - CV Accuracy: 0.5027 ± 0.0166 | Precision: 0.5040 ± 0.0273 | Recall: 0.2993 ± 0.0364 | F1: 0.3748 ± 0.0338
Saved final CV model to saved_models/Bigram_CV_nb_pipeline.pkl
Trigram - CV Accuracy: 0.5064 ± 0.0209 | Precision: 0.5077 ± 0.0274 | Recall: 0.4083 ± 0.0394 | F1: 0.4520 ± 0.0324
Saved final CV model to saved_models/Trigram_CV_nb_pipeline.pkl


In [59]:
# Directory where models are saved
save_dir = "saved_models"

# Reload all Naive Bayes models into a new dictionary
loaded_pipelines = {}

for name in embedding_experiments.keys():
    model_variants = {}

    # Load train/test split NB model
    path_train = os.path.join(save_dir, f"{name}_nb_pipeline.pkl")
    if os.path.exists(path_train):
        model_variants['train'] = joblib.load(path_train)
        print(f"Loaded trained NB model: {name}")

    # Load final CV NB model
    path_cv = os.path.join(save_dir, f"{name}_CV_nb_pipeline.pkl")
    if os.path.exists(path_cv):
        model_variants['cv'] = joblib.load(path_cv)
        print(f"Loaded CV NB model: {name}")

    if model_variants:
        loaded_pipelines[name] = model_variants


Loaded trained NB model: POS
Loaded CV NB model: POS
Loaded trained NB model: Unigram
Loaded CV NB model: Unigram
Loaded trained NB model: Bigram
Loaded CV NB model: Bigram
Loaded trained NB model: Trigram
Loaded CV NB model: Trigram


In [60]:
# Load saved Naive Bayes pipelines
save_dir = "saved_models"
model_names = ["POS", "Unigram", "Bigram", "Trigram"]

# Stub feature-extractor functions
def get_pos_features(sentence):
    return np.random.rand(50)

def get_unigram_features(sentence):
    return np.random.rand(100)

def get_bigram_features(sentence):
    return np.random.rand(50)

def get_trigram_features(sentence):
    return np.random.rand(50)

def extract_features_for_models(sentence):
    return {
        'POS':     get_pos_features(sentence),
        'Unigram': get_unigram_features(sentence),
        'Bigram':  get_bigram_features(sentence),
        'Trigram': get_trigram_features(sentence)
    }

# Prediction function
def predict_sentence(sentence, pipelines):
    feats = extract_features_for_models(sentence)
    print(f"\nInput sentence: {sentence}\n")
    for name, variants in pipelines.items():
        arr = pd.DataFrame({'feat': [feats[name]]})
        if 'train' in variants:
            pred = variants['train'].predict(arr)[0]
            prob = variants['train'].predict_proba(arr)[0][1]
            label = "Correct" if pred == 1 else "Incorrect"
            print(f"{name} (train): {label} (confidence: {prob:.2f})")
        if 'cv' in variants:
            pred = variants['cv'].predict(arr)[0]
            prob = variants['cv'].predict_proba(arr)[0][1]
            label = "Correct" if pred == 1 else "Incorrect"
            print(f"{name} (CV):    {label} (confidence: {prob:.2f})")

# Run interactive prompt
if __name__ == "__main__":
    sentence = input("Enter a sentence to check: ")
    predict_sentence(sentence, loaded_pipelines)

Enter a sentence to check: she are running now

Input sentence: she are running now

POS (train): Incorrect (confidence: 0.00)
POS (CV):    Incorrect (confidence: 0.00)
Unigram (train): Correct (confidence: 1.00)
Unigram (CV):    Incorrect (confidence: 0.00)
Bigram (train): Correct (confidence: 1.00)
Bigram (CV):    Correct (confidence: 1.00)
Trigram (train): Correct (confidence: 1.00)
Trigram (CV):    Correct (confidence: 1.00)


#SVM

In [61]:
from sklearn.svm import SVC

# Make directory to save models
save_dir = "saved_models"
os.makedirs(save_dir, exist_ok=True)

def stack_feats(X):
    return np.vstack(X['feat'])

def make_embedding_pipeline_svm(ug_col, st_col):
    n = len(df)
    feats = list(df[ug_col]) + list(df[st_col])
    y     = np.array([0]*n + [1]*n)  # 0=ungrammatical, 1=correct
    X_df  = pd.DataFrame({'feat': feats})
    stacker = FunctionTransformer(stack_feats, validate=False)

    pipe = Pipeline([
        ('stack', stacker),
        ('scaler', StandardScaler()),
        ('clf',   SVC(kernel='linear',class_weight='balanced',probability=True))
    ])
    return pipe, X_df, y

trained_pipelines = {}
print("=== Train/Test split 90/10 with SVM ===")
for name, (ug, st) in embedding_experiments.items():
    pipe, X_df, y = make_embedding_pipeline_svm(ug, st)

    X_tr, X_te, y_tr, y_te = train_test_split(
        X_df, y,
        test_size=0.1,
        stratify=y,
        random_state=42
    )
    pipe.fit(X_tr, y_tr)
    preds = pipe.predict(X_te)
    trained_pipelines[name] = pipe

    print(f"\n{name} accuracy: {accuracy_score(y_te, preds):.4f}")
    print(classification_report(y_te, preds, digits=4))

    # Save model
    model_path = os.path.join(save_dir, f"{name}_svm_pipeline.pkl")
    joblib.dump(pipe, model_path)
    print(f"Saved model to {model_path}")

=== Train/Test split 90/10 with SVM ===

POS accuracy: 0.5495
              precision    recall  f1-score   support

           0     0.5549    0.5000    0.5260       202
           1     0.5450    0.5990    0.5708       202

    accuracy                         0.5495       404
   macro avg     0.5500    0.5495    0.5484       404
weighted avg     0.5500    0.5495    0.5484       404

Saved model to saved_models/POS_svm_pipeline.pkl

Unigram accuracy: 0.5470
              precision    recall  f1-score   support

           0     0.5450    0.5693    0.5569       202
           1     0.5492    0.5248    0.5367       202

    accuracy                         0.5470       404
   macro avg     0.5471    0.5470    0.5468       404
weighted avg     0.5471    0.5470    0.5468       404

Saved model to saved_models/Unigram_svm_pipeline.pkl

Bigram accuracy: 0.4827
              precision    recall  f1-score   support

           0     0.4817    0.4554    0.4682       202
           1     0.483

In [62]:
print("\n=== 10-Fold Cross Validation with SVM  ===")
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

cv_results = {}

for name, (ug, st) in embedding_experiments.items():
    # Prepare input features and labels
    pipe, X_df, y = make_embedding_pipeline_svm(ug, st)

    fold_precisions = []
    fold_recalls = []
    fold_f1s = []
    all_preds = []
    all_true = []
    fold_accuracies = []

    for fold, (train_idx, val_idx) in enumerate(cv.split(X_df, y)):
        X_train, X_val = X_df.iloc[train_idx], X_df.iloc[val_idx]
        y_train, y_val = y[train_idx], y[val_idx]

        fold_model = Pipeline([
            ('stack', FunctionTransformer(stack_feats, validate=False)),
            ('clf', SVC(kernel='linear', class_weight='balanced', probability=True))
        ])
        fold_model.fit(X_train, y_train)
        preds = fold_model.predict(X_val)

        all_preds.extend(preds)
        all_true.extend(y_val)

        # Metrics per fold
        acc = accuracy_score(y_val, preds)
        prec = precision_score(y_val, preds, average='binary', zero_division=0)
        rec = recall_score(y_val, preds, average='binary', zero_division=0)
        f1 = f1_score(y_val, preds, average='binary', zero_division=0)

        fold_accuracies.append(acc)
        fold_precisions.append(prec)
        fold_recalls.append(rec)
        fold_f1s.append(f1)

    # Mean and Std for each metric
    mean_acc, std_acc = np.mean(fold_accuracies), np.std(fold_accuracies)
    mean_prec, std_prec = np.mean(fold_precisions), np.std(fold_precisions)
    mean_rec, std_rec = np.mean(fold_recalls), np.std(fold_recalls)
    mean_f1, std_f1 = np.mean(fold_f1s), np.std(fold_f1s)

    # Print summary in desired format
    print(f"{name} - CV Accuracy: {mean_acc:.4f} ± {std_acc:.4f} | "
          f"Precision: {mean_prec:.4f} ± {std_prec:.4f} | "
          f"Recall: {mean_rec:.4f} ± {std_rec:.4f} | "
          f"F1: {mean_f1:.4f} ± {std_f1:.4f}")

    # Save the final model trained on the entire dataset
    pipe.fit(X_df, y)
    model_path = os.path.join(save_dir, f"{name}_CV_svm_pipeline.pkl")
    joblib.dump(pipe, model_path)
    print(f"Saved final CV model to {model_path}")


=== 10-Fold Cross Validation with SVM  ===
POS - CV Accuracy: 0.5317 ± 0.0172 | Precision: 0.5286 ± 0.0153 | Recall: 0.5887 ± 0.0249 | F1: 0.5568 ± 0.0169
Saved final CV model to saved_models/POS_CV_svm_pipeline.pkl
Unigram - CV Accuracy: 0.5188 ± 0.0281 | Precision: 0.5180 ± 0.0266 | Recall: 0.5516 ± 0.0437 | F1: 0.5336 ± 0.0298
Saved final CV model to saved_models/Unigram_CV_svm_pipeline.pkl
Bigram - CV Accuracy: 0.5069 ± 0.0173 | Precision: 0.4045 ± 0.2043 | Recall: 0.3837 ± 0.4234 | F1: 0.3024 ± 0.2913
Saved final CV model to saved_models/Bigram_CV_svm_pipeline.pkl
Trigram - CV Accuracy: 0.4936 ± 0.0090 | Precision: 0.3561 ± 0.1898 | Recall: 0.3064 ± 0.3814 | F1: 0.2587 ± 0.2596
Saved final CV model to saved_models/Trigram_CV_svm_pipeline.pkl


In [63]:
# Reload all models into a new dictionary
loaded_pipelines = {}
for name in embedding_experiments.keys():
    model_variants = {}
    # Load train/test split model
    path_train = os.path.join(save_dir, f"{name}_svm_pipeline.pkl")
    if os.path.exists(path_train):
        model_variants['train'] = joblib.load(path_train)
        print(f"Loaded trained model: {name}")

    # Load final CV model
    path_cv = os.path.join(save_dir, f"{name}_CV_svm_pipeline.pkl")
    if os.path.exists(path_cv):
        model_variants['cv'] = joblib.load(path_cv)
        print(f"Loaded CV model: {name}")

    # Optional: use loaded_pipelines as default in your prediction functions
    loaded_pipelines[name] = model_variants


Loaded trained model: POS
Loaded CV model: POS
Loaded trained model: Unigram
Loaded CV model: Unigram
Loaded trained model: Bigram
Loaded CV model: Bigram
Loaded trained model: Trigram
Loaded CV model: Trigram


In [64]:
def get_pos_features(sentence):
    return np.random.rand(50)

def get_unigram_features(sentence):
    return np.random.rand(100)

def get_bigram_features(sentence):
    return np.random.rand(50)

def get_trigram_features(sentence):
    return np.random.rand(50)

def extract_features_for_models(sentence):
    features = {}

    features['POS'] = get_pos_features(sentence)
    features['Unigram'] = get_unigram_features(sentence)
    features['Bigram'] = get_bigram_features(sentence)
    features['Trigram'] = get_trigram_features(sentence)

    return features

def predict_sentence(sentence, pipelines):
    feats_dict = extract_features_for_models(sentence)
    print(f"Input sentence: {sentence}\n")

    for model_name, model_variants in pipelines.items():
        X_test_df = pd.DataFrame({'feat': [feats_dict[model_name]]})

        # Predict using train/test model
        if 'train' in model_variants:
            pred = model_variants['train'].predict(X_test_df)[0]
            prob = model_variants['train'].predict_proba(X_test_df)[0][1]
            print(f"{model_name} (train): {'Correct' if pred == 1 else 'Incorrect'} (confidence: {prob:.2f})")

        # Predict using CV model
        if 'cv' in model_variants:
            pred = model_variants['cv'].predict(X_test_df)[0]
            prob = model_variants['cv'].predict_proba(X_test_df)[0][1]
            print(f"{model_name} (CV): {'Correct' if pred == 1 else 'Incorrect'} (confidence: {prob:.2f})")


# Example usage:
sentence = input("Enter a sentence to check: ")
predict_sentence(sentence, loaded_pipelines)


Enter a sentence to check: she are running now
Input sentence: she are running now

POS (train): Correct (confidence: 1.00)
POS (CV): Correct (confidence: 1.00)
Unigram (train): Incorrect (confidence: 0.47)
Unigram (CV): Incorrect (confidence: 0.00)
Bigram (train): Incorrect (confidence: 0.47)
Bigram (CV): Incorrect (confidence: 0.93)
Trigram (train): Incorrect (confidence: 0.78)
Trigram (CV): Incorrect (confidence: 0.54)


# XGBoost

In [65]:
from xgboost import XGBClassifier

# Configuration
save_dir = "saved_models"
os.makedirs(save_dir, exist_ok=True)

# 0) Parse embeddings (handle both strings and arrays)
def parse_embedding(s):
    if isinstance(s, str):
        return np.array([float(x) for x in s.strip("[]").split()], dtype=float)
    elif isinstance(s, (list, np.ndarray)):
        return np.array(s, dtype=float)
    else:
        return np.zeros(0, dtype=float)

for ug_col, st_col in embedding_experiments.values():
    df[ug_col] = df[ug_col].apply(parse_embedding)
    df[st_col] = df[st_col].apply(parse_embedding)

# Helper to stack feature arrays
def stack_feats(X):
    return np.vstack(X["feat"])

# Pipeline builder for XGBoost
def make_embedding_pipeline_xgb(ug_col, st_col):
    n     = len(df)
    feats = list(df[ug_col]) + list(df[st_col])
    y     = np.array([0]*n + [1]*n)
    X_df  = pd.DataFrame({"feat": feats})

    stacker = FunctionTransformer(stack_feats, validate=False)
    pipe    = Pipeline([
        ("stack", stacker),
        ("clf",   XGBClassifier(eval_metric="logloss", random_state=42))
    ])
    return pipe, X_df, y

# 2) 90/10 Train/Test Split
print("=== Train/Test split 90/10 with XGBoost ===")
for name, (ug, st) in embedding_experiments.items():
    pipe, X_df, y = make_embedding_pipeline_xgb(ug, st)
    X_tr, X_te, y_tr, y_te = train_test_split(
        X_df, y, test_size=0.1, stratify=y, random_state=42
    )

    pipe.fit(X_tr, y_tr)
    preds = pipe.predict(X_te)

    print(f"\n{name} accuracy: {accuracy_score(y_te, preds):.4f}")
    print(classification_report(y_te, preds, digits=4))

    # Save the pipeline
    model_path = os.path.join(save_dir, f"{name}_xgb_pipeline.pkl")
    joblib.dump(pipe, model_path)
    print(f"Saved XGBoost pipeline to {model_path}")


=== Train/Test split 90/10 with XGBoost ===

POS accuracy: 0.4827
              precision    recall  f1-score   support

           0     0.4834    0.5050    0.4939       202
           1     0.4819    0.4604    0.4709       202

    accuracy                         0.4827       404
   macro avg     0.4826    0.4827    0.4824       404
weighted avg     0.4826    0.4827    0.4824       404

Saved XGBoost pipeline to saved_models/POS_xgb_pipeline.pkl

Unigram accuracy: 0.4406
              precision    recall  f1-score   support

           0     0.4400    0.4356    0.4378       202
           1     0.4412    0.4455    0.4433       202

    accuracy                         0.4406       404
   macro avg     0.4406    0.4406    0.4406       404
weighted avg     0.4406    0.4406    0.4406       404

Saved XGBoost pipeline to saved_models/Unigram_xgb_pipeline.pkl

Bigram accuracy: 0.3911
              precision    recall  f1-score   support

           0     0.3911    0.3911    0.3911       

In [67]:
import os
import joblib
from sklearn.model_selection import GridSearchCV, StratifiedKFold

# 10-fold stratified CV splitter
cv_outer = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# Hyperparameter grid
param_grid = {
    "clf__n_estimators":  [50, 100, 200],
    "clf__max_depth":     [3, 4, 5],
    "clf__learning_rate": [0.01, 0.05, 0.1]
}

print("\n=== 10-Fold Grid Search CV for XGBoost ===")
for name, (ug, st) in embedding_experiments.items():
    # rebuild the pipeline and data
    pipe, X_df, y = make_embedding_pipeline_xgb(ug, st)

    grid = GridSearchCV(
        estimator=pipe,
        param_grid=param_grid,
        cv=cv_outer,
        scoring="accuracy",
        n_jobs=-1,
        verbose=2
    )
    grid.fit(X_df, y)

    # report
    print(f"\n{name} best params: {grid.best_params_}")
    print(f"{name} best CV accuracy: {grid.best_score_:.4f}")

    # save the tuned pipeline
    best_path = os.path.join(save_dir, f"{name}_xgb_grid_pipeline.pkl")
    joblib.dump(grid.best_estimator_, best_path)
    print(f"Saved best XGBoost pipeline to {best_path}")


=== 10-Fold Grid Search CV for XGBoost ===
Fitting 10 folds for each of 27 candidates, totalling 270 fits

POS best params: {'clf__learning_rate': 0.05, 'clf__max_depth': 3, 'clf__n_estimators': 50}
POS best CV accuracy: 0.5339
Saved best XGBoost pipeline to saved_models/POS_xgb_grid_pipeline.pkl
Fitting 10 folds for each of 27 candidates, totalling 270 fits

Unigram best params: {'clf__learning_rate': 0.01, 'clf__max_depth': 3, 'clf__n_estimators': 100}
Unigram best CV accuracy: 0.5300
Saved best XGBoost pipeline to saved_models/Unigram_xgb_grid_pipeline.pkl
Fitting 10 folds for each of 27 candidates, totalling 270 fits

Bigram best params: {'clf__learning_rate': 0.05, 'clf__max_depth': 3, 'clf__n_estimators': 50}
Bigram best CV accuracy: 0.4950
Saved best XGBoost pipeline to saved_models/Bigram_xgb_grid_pipeline.pkl
Fitting 10 folds for each of 27 candidates, totalling 270 fits

Trigram best params: {'clf__learning_rate': 0.01, 'clf__max_depth': 4, 'clf__n_estimators': 50}
Trigram b

In [68]:
# Reload saved XGBoost pipelines
save_dir = "saved_models"
loaded_xgb = {}
for name in embedding_experiments.keys():
    variants = {}
    p_train = os.path.join(save_dir, f"{name}_xgb_pipeline.pkl")
    p_grid  = os.path.join(save_dir, f"{name}_xgb_grid_pipeline.pkl")
    if os.path.exists(p_train):
        variants['train'] = joblib.load(p_train)
        print(f"Loaded XGB train model: {name}")
    if os.path.exists(p_grid):
        variants['grid']  = joblib.load(p_grid)
        print(f"Loaded XGB grid-model: {name}")
    if variants:
        loaded_xgb[name] = variants

Loaded XGB train model: POS
Loaded XGB grid-model: POS
Loaded XGB train model: Unigram
Loaded XGB grid-model: Unigram
Loaded XGB train model: Bigram
Loaded XGB grid-model: Bigram
Loaded XGB train model: Trigram
Loaded XGB grid-model: Trigram


In [69]:
# Stub feature-extractor functions
def get_pos_features(sentence):
    return np.random.rand(50)
def get_unigram_features(sentence):
    return np.random.rand(100)
def get_bigram_features(sentence):
    return np.random.rand(50)
def get_trigram_features(sentence):
    return np.random.rand(50)

def extract_features_for_models(sentence):
    return {
        'POS':     get_pos_features(sentence),
        'Unigram': get_unigram_features(sentence),
        'Bigram':  get_bigram_features(sentence),
        'Trigram': get_trigram_features(sentence)
    }

# Prediction function for XGB only
def predict_with_xgb(sentence, xgb_pipelines):
    feats = extract_features_for_models(sentence)
    print(f"\nInput sentence: {sentence}\n")
    for name, variants in xgb_pipelines.items():
        df_feat = pd.DataFrame({'feat': [feats[name]]})
        for tag, model in variants.items():
            pred = model.predict(df_feat)[0]
            prob = model.predict_proba(df_feat)[0][1]
            label = "Correct" if pred == 1 else "Incorrect"
            print(f"{name} XGB ({tag}): {label} (confidence: {prob:.2f})")

# Test on an example
sentence = input("Enter a sentence to check:\n> ")
predict_with_xgb(sentence, loaded_xgb)

Enter a sentence to check:
> she are running now

Input sentence: she are running now

POS XGB (train): Correct (confidence: 0.65)
POS XGB (grid): Incorrect (confidence: 0.34)
Unigram XGB (train): Incorrect (confidence: 0.12)
Unigram XGB (grid): Incorrect (confidence: 0.38)
Bigram XGB (train): Incorrect (confidence: 0.22)
Bigram XGB (grid): Incorrect (confidence: 0.35)
Trigram XGB (train): Incorrect (confidence: 0.34)
Trigram XGB (grid): Correct (confidence: 0.53)
